# we will use [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/) packages to help us implement graph neural networks. 

## Setup

In [1]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch-geometric

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install networkx matplotlib

Note: you may need to restart the kernel to use updated packages.


## Import Necessary Libraries
PyTorch Geometric has two classes for storing and/or transforming graphs into tensor format. One is `torch_geometric.datasets`, which contains a variety of common graph datasets. Another is `torch_geometric.data`, which provides the data handling of graphs in PyTorch tensors.


In [1]:
# Import necessary libraries
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GCNConv, BatchNorm
from torch_geometric.loader import DataLoader

## Load the Dataset
ENZYMES is a dataset of 600 protein tertiary structures obtained from the BRENDA enzyme database. 
Each graph in the ENZYMES dataset corresponds to a protein, and proteins are classified into one of 6 EC (Enzyme Commission) top-level classes. These classes are based on the type of chemical reaction the enzyme catalyzes.

In [2]:
# Load the ENZYMES dataset
dataset = TUDataset(root='/tmp/enzymes', name='ENZYMES')

Processing...
Done!


## What is the number of classes and number of features in the ENZYMES dataset? 

In [3]:
def basic_info(dataset):
    # TODO: Implement a function that takes a PyG dataset object
    # and returns the number of classes, number of features for that dataset.
    num_class = 0
    num_features = 0

    ############# Your code here ############
    ## (~2 lines of code)
    num_class = dataset.num_classes
    num_features = dataset.num_features
    #########################################

    return num_class, num_features

num_classes, num_features = basic_info(dataset)
print("ENZYMES dataset has {} classes".format(num_classes))
print("ENZYMES dataset has {} features".format(num_features))

ENZYMES dataset has 6 classes
ENZYMES dataset has 3 features


## What is the label of the graph with index 100 in the ENZYMES dataset?
Each PyG dataset stores a list of `torch_geometric.data.Data` objects, where each `torch_geometric.data.Data` object represents a graph. We can easily get the `Data` object by indexing into the dataset.

For more information such as what is stored in the `Data` object, please refer to the [documentation](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.Data.html#torch_geometric.data.Data).

In [4]:
def get_graph_class(dataset, idx):
    # TODO: Implement a function that takes a PyG dataset object,
    # an index of a graph within the dataset, and returns the class/label
    # of the graph (as an integer).

    label = -1

    ############# Your code here ############
    ## (~1 line of code)
    label = int(dataset[idx].y)
    #########################################

    return label
    
idx = 100
single_graph = dataset[idx]
label = get_graph_class(dataset, idx)
print(single_graph)
print('Graph with index {} has label {}'.format(idx, label))

Data(edge_index=[2, 176], x=[45, 3], y=[1])
Graph with index 100 has label 4


## How many edges does the graph with index 100 have?

In [5]:
def get_graph_num_edges(dataset, idx):
    # TODO: Implement a function that takes a PyG dataset object,
    # the index of a graph in the dataset, and returns the number of
    # edges in the graph (as an integer). You should not count an edge
    # twice if the graph is undirected. For example, in an undirected
    # graph G, if two nodes v and u are connected by an edge, this edge
    # should only be counted once.

    num_edges = 0

    ############# Your code here ############
    ## Note:
    ## 1. Do not return the data.num_edges directly
    ## 2. For undirected graphs, each edge is stored twice (once in each direction).
    ## 3. Look at the PyG dataset built in functions
    ## (~4 lines of code)

    if dataset[idx].is_undirected():
        num_edges = int(dataset[idx].num_edges/2)
    else:
        num_edges = dataset[idx].num_edges

    #########################################

    return num_edges

idx = 100
num_edges = get_graph_num_edges(dataset, idx)
print('Graph with index {} has {} edges'.format(idx, num_edges))
print(int(dataset[idx].edge_index.shape[1]/2) == num_edges)
# It's True because edge_index stores the bi-directional edge information, for example an undirected edege between node 2 and node 5 
# is stored twice by [2, 5] and [5, 2] in edge_index of an Data object. 

Graph with index 100 has 88 edges
True


## Split data using inductive setting
Randomly divide the dataset into training set and test set using inductive setting where 80 percent of the whole dataset are in training set and the rest are divided into test dataset.

Each datepoint in the dataset is a graph. So it is easier for us to wrap datasets using [`torch_geometric.loader.DataLoader`](https://pytorch-geometric.readthedocs.io/en/latest/modules/loader.html#torch_geometric.loader.DataLoader)' so as to be able to sample minibatches during training. 

In [6]:
from torch_geometric.loader import DataLoader

def inductive_split(dataset, split_ratio, batch_size):
    # TODO: Implement a function to split the dataset into training dataset and test dataset
    # using inductive setting. 
    # dataset: The PyTorch Geometric graph dataset.
    # split_ratio(float): Ratio of nodes to be used for training.
    # batch_size(int): batch_size for the dataloader

    ############# Your code here ############
    # Shuffle the dataset
    dataset = dataset.shuffle()
    
    # Split the dataset into train and test sets
    size = int(split_ratio*len(dataset))
    train_dataset = dataset[:size]
    test_dataset = dataset[size:]
    
    # Wrap the datset using Dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    #########################################

    return train_loader, test_loader
    
train_loader, test_loader = inductive_split(dataset, split_ratio=0.8, batch_size=64)

## Build GCN Model
In this exercise, we will implement **batch normalization, dropout, global pooling, and skip layer connection**. 

Implement your GCN model following the structure:

input --> GCN layer1 --> Batch Normalization --> non-linear activation --> dropout --> GCN layer2 --> Batch Normalization --> non-linear activation --> dropout --> graph-level prediction head(global pooling) --> prediction MLP

For initial input, also skip connections for the first GCN layer, that is, in addition to the above structure, you also have:
input --> GCN layer2 --> Batch Normalization --> non-linear activation --> dropout --> graph-level prediction head(gloabl pooling) --> prediction MLP

In [17]:
from torch_geometric.nn import GCNConv, BatchNorm
from torch_geometric.nn import global_add_pool, global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, input_channels, hidden_channels, output_channels, dropout):
        super(GCN, self).__init__()

        # TODO: Implement a function that initializes GCNconv layers and batch normalizations
        ############# Your code here ############
        ## Note:
        ## 1. The parameters you can set for GCNConv include 'in_channels' and
        ## 'out_channels'. For more information please refer to the documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv
        ## 2. The only parameter you need to set for BatchNorm is 'num_features'
        ## For more information please refer to the documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.norm.BatchNorm.html

        self.conv1 = GCNConv(input_channels, input_channels)
        self.batch_norm1 = BatchNorm(input_channels)
        
        self.conv2 = GCNConv(input_channels, hidden_channels)
        self.batch_norm2 = BatchNorm(hidden_channels)

        self.out = torch.nn.Linear(hidden_channels, output_channels)
        
        self.dropout = torch.nn.Dropout(p=dropout)
        
        #########################################


    def forward(self, data):
        # TODO: Implement a function that takes the feature tensor x and
        # edge_index tensor and batch_index tensor and returns the prediction tensor 

        x = data.x
        edge_index = data.edge_index
        batch = data.batch

        out = None

        ############# Your code here ############
        ## 1. torch.nn.functional.relu and torch.nn.functional.dropout are useful
        ## For more information please refer to the documentation:
        ## https://pytorch.org/docs/stable/nn.functional.html
        ## 2. We already import torch.nn.functional as F
        ## 3. For initial input, also skip connection for the first GCN layer. 
        ## 4. Choose your prefered Global Pooling over the nodes to create graph level embeddings that can be used to
        ## predict properties for the each graph. Remeber that the batch attribute will be essential 
        ## for performining Global Pooling over our mini-batch of graphs.

        duplicate_x = x
        
        ## First GCN layer
        x = self.conv1(x, edge_index)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.dropout(x)
        duplicate_x_2 = x
        ## Skip connection
        x = duplicate_x + x
        
        ## Second GCN layer
        x = self.conv1(x, edge_index)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.dropout(x)
        # print(x.shape)
        ## Skip connection
        x = duplicate_x_2 + x

        ## Second GCN layer
        x = self.conv2(x, edge_index)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.dropout(x)

        ## Global Pooling 
        ## (~1 line of code)
        x = global_mean_pool(x, batch)

        ## Output layer
        out = self.out(x)
        
        #########################################

        return out

## Training your model
Classifying enzyme dataset is a hard task. So with a simple model like ours, the accuracy is not very high.

You can refer to this https://paperswithcode.com/sota/graph-classification-on-enzymes website for accuracies of classifying enzyme dataset from different research papers. 

In [18]:
hidden_channels = 32
GCNmodel = GCN(dataset.num_node_features, hidden_channels, dataset.num_classes, 0.5)

# Define the optimizers for the models
GCNoptimizer = torch.optim.Adam(GCNmodel.parameters(), lr=0.05)


# Function to calculate accuracy
def accuracy(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        pred = model(data)
        pred = pred.argmax(dim=1)
        correct += int(pred.eq(data.y).sum().item())
    accuracy = correct / len(loader.dataset)
    return accuracy

# Training Loop
def train(model, optimizer):
    model.train()
    Tloss = 0

    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.cross_entropy(out, data.y)
        loss.backward()
        optimizer.step()
        Tloss += loss.item() * data.num_graphs

    return Tloss / len(train_loader.dataset)

# Run the training and testing for GCN Model
for epoch in range(50):
    loss = train(GCNmodel, GCNoptimizer)
    train_acc = accuracy(GCNmodel, train_loader)
    test_acc = accuracy(GCNmodel, test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 000, Loss: 1.8657, Train Acc: 0.2354, Test Acc: 0.2250
Epoch: 001, Loss: 1.7811, Train Acc: 0.2292, Test Acc: 0.2250
Epoch: 002, Loss: 1.7513, Train Acc: 0.2229, Test Acc: 0.1833
Epoch: 003, Loss: 1.7452, Train Acc: 0.2375, Test Acc: 0.2250
Epoch: 004, Loss: 1.7358, Train Acc: 0.2479, Test Acc: 0.2083
Epoch: 005, Loss: 1.7519, Train Acc: 0.2229, Test Acc: 0.2583
Epoch: 006, Loss: 1.7053, Train Acc: 0.2083, Test Acc: 0.1833
Epoch: 007, Loss: 1.7473, Train Acc: 0.2500, Test Acc: 0.2250
Epoch: 008, Loss: 1.7153, Train Acc: 0.1979, Test Acc: 0.1833
Epoch: 009, Loss: 1.7334, Train Acc: 0.1875, Test Acc: 0.2167
Epoch: 010, Loss: 1.7469, Train Acc: 0.2542, Test Acc: 0.2333
Epoch: 011, Loss: 1.7275, Train Acc: 0.2875, Test Acc: 0.2500
Epoch: 012, Loss: 1.7142, Train Acc: 0.2625, Test Acc: 0.2417
Epoch: 013, Loss: 1.7040, Train Acc: 0.2542, Test Acc: 0.2750
Epoch: 014, Loss: 1.7071, Train Acc: 0.2792, Test Acc: 0.2417
Epoch: 015, Loss: 1.7220, Train Acc: 0.2396, Test Acc: 0.2417
Epoch: 0

## Data Split Transductive Setting

In a transductive setting, all nodes are used during both training and testing, but their labels are only revealed according to the split. 

A simple way to split your data transductively using masks to indicate which nodes belong to the training, validation, and test sets.

Implement the transductive_split function below which takes a graph data object, splits the nodes into training, validation, and test sets according to the specified ratios, and adds boolean masks to the data object indicating whether each node is in the training, validation, or test set. 

In a transductive setting, all node features (data.x) are available during training, but the labels (data.y) are only revealed according to these masks.

In [19]:
def transductive_split(data, train_ratio=0.6, val_ratio=0.2, seed=42):
    
    # Perform a transductive split on a PyTorch Geometric graph data.
    
    # Parameters:
    # data (Data): The PyTorch Geometric Data object representing the graph.
    # train_ratio (float): Ratio of nodes to be used for training.
    # val_ratio (float): Ratio of nodes to be used for validation.
    # seed (int): Random seed for reproducibility.

    # test_ratio is omitted because it should equal to 1 - train_ratio - val_ratio

    # Returns:
    # data (Data): The PyTorch Geometric Data object with added train_mask, val_mask, and test_mask attributes.

    torch.manual_seed(seed)
    
    train_mask = None
    val_mask = None
    test_mask = None
    
    ############# Your code here ############
    # Number of nodes in the graph
    num_nodes = data.num_nodes

    # Indices of all nodes shuffled
    indices = torch.randperm(num_nodes)
    print (indices)

    # Determine the split sizes
    train_size = int(num_nodes * train_ratio)
    val_size = int(num_nodes * val_ratio)

    # Create masks indicating whether a node belongs to the train, validation, or test set
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    print (train_mask)

    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True
    
    #########################################
    
    # Add masks to data
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

    return data

# Example usage:
# Assume 'data' is your PyTorch Geometric Data object
# data = Data(x=node_features, edge_index=edge_indices, y=node_labels)  # Example initialization

# Perform the transductive split
# data = transductive_split(data, train_ratio=0.6, val_ratio=0.2)

# Now 'data' contains 'train_mask', 'val_mask', and 'test_mask' which can be used to mask the nodes for training, validation, and testing.

In [20]:
from torch_geometric.datasets import KarateClub

# Load the Karate Club dataset
dataset = KarateClub()
data = dataset[0]  # Get the graph Data object

# Perform the transductive split
data = transductive_split(data, train_ratio=0.6, val_ratio=0.2)

print(data.train_mask)
print(data.val_mask)
print(data.test_mask)
# count the number of True and False in the mask attributes. Do they match the ratios? You can also play with the radios.
# "True" basically means the node's label is fed into training/validation/test.

tensor([30,  0,  1, 17, 10,  6, 18,  2, 26, 22, 20, 24,  8, 31, 25,  9, 27, 16,
        14,  4, 21, 15, 23, 29, 33, 19, 13, 28,  5, 11,  7, 12,  3, 32])
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False])
tensor([ True,  True,  True, False,  True, False,  True, False,  True,  True,
         True, False, False, False,  True, False,  True,  True,  True, False,
         True, False,  True, False,  True,  True,  True,  True, False, False,
         True,  True, False, False])
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False,  True,
        False,  True, False,  True, False, False, False, False, False,  True,
        False, False, False,  True])
tensor([False, False, False,  True

## Graph Manipulation
In this exercise, we use the KarateClub data as an example to perform the following tasks.

Tasks:
1. constant node features or one-hot node features?
2. Add virtual node / edge (for sparse graph)
3. Sample subgraph / neighours (for dense / large graph)

In [21]:
# node features
# get the node features of a graph dataset, are the node features constant node features or one-hot node features
def check_node_features(data):

    ############# Your code here ############
    # get node feature matrix
    x = data.x 

    print (torch.all(x == x[0], dim=0))
    
    # Check if features are constant
    if torch.all(x == x[0], dim=0).all():
        return "The node features are constant across all nodes."

    # Check if features are one-hot node features (each node has a unique one-hot id)
    # One-Hot Criterion: Check if each row sums to 1 and contains exactly one '1'
    row_sums = torch.sum(x, dim=1)
    one_hot_criterion = torch.all(row_sums == 1) and torch.all((x == 0) | (x == 1)) ## each row sums to 1 and 
                                                                                    ## the matrix only contains 0s and 1s
    print (one_hot_criterion)
    
    # Uniqueness Criterion: Check if all rows are unique
    # we sum up the columns, each column should also sum to 1 if all rows are unique
    col_sums = torch.sum(x, dim=0)
    uniqueness_criterion = torch.all(col_sums == 1)
    if one_hot_criterion and uniqueness_criterion:
        return "The node features are one-hot encoded."

    #########################################

    return "The node features are neither constant across all nodes nor one-hot encoded."

In [22]:
# Load the Karate Club dataset
dataset = KarateClub()
data = dataset[0]  # Get the graph Data object

check_node_features(data)

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False])
tensor(True)


'The node features are one-hot encoded.'

In [23]:
# Add virtual Edge/nodes
def add_virtual_node(data):
    # data (Data): The PyTorch Geometric Data object representing the graph.
    
    ############# Your code here ############
    num_nodes = data.num_nodes

    # Add a virtual node to the node feature matrix
    # 1. generate the virtual node features as the mean of exisiting features.
    # 2. add the virtual node features to the existing node feature matrix. 
    
    virtual_node_features = torch.mean(data.x, dim=0, keepdim=True)  # Example: using the mean of existing features
    data.x = torch.cat([data.x, virtual_node_features], dim=0)

    # Connect the virtual node to all existing nodes
    # Assume all edges are undirected
    # 1. create the tensor for connecting edges
    # 2. making the edge undirected as how the Pytorch Geometric Data object would represent a undirected edge
   
    virtual_node_edges = torch.tensor([[num_nodes] * num_nodes, list(range(num_nodes))], dtype=torch.long)
    print (virtual_node_edges)
    virtual_node_edges = torch.cat([virtual_node_edges, virtual_node_edges.flip(0)], dim=1)  # Making it undirected

    # Add these edges to the edge index
    data.edge_index = torch.cat([data.edge_index, virtual_node_edges], dim=1)
    
    #########################################
    
    return data

In [30]:
# Load the Karate Club dataset
dataset = KarateClub()
data = dataset[0]  # Get the graph Data object
print(data.edge_index.shape)
data=add_virtual_node(data)
print(data.edge_index.shape)
# You should find out that the added node is connected to all the existing nodes. 
# Understanding edge_index: edge_index[0][i] is connected to edge_index[1][i] 

torch.Size([2, 156])
tensor([[34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,
         34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]])
torch.Size([2, 224])


In [31]:
# Sample neighours for a given node 
import random 

def sample_neighbors(data, node_id, sample_neigh_num):
    # data (Data): The PyTorch Geometric Data object representing the graph.
    # node_id (int): The ID of the node for which to find neighbors.
    # sample_neigh_num(int): The number of neighbours to sample each time. 
    
    ############# Your code here ############ 
    
    # Find all the neighours of the specified node
    all_neighbours = []
    for idx in range(len(data.edge_index[0])):
        node = data.edge_index[0][idx]
        if node == node_id:
            neighbor = int(data.edge_index[1][idx])
            all_neighbours.append(neighbor)

    print (all_neighbours)
    
    # Randomly sample neighbours from all neighbours
    # return the sampled neighbour nodes
    if sample_neigh_num >= len(all_neighbours): ## do not have enough neighbours to sample. Keep all neigbour nodes
        sampled = all_neighbours
    else:
        sampled = random.sample(all_neighbours, sample_neigh_num)
    
    #########################################
    
    return sampled

In [32]:
dataset = KarateClub()
data = dataset[0]

# Specify the node for which to sample neighbors
node_id = 0  # For example, node 0

# Sample 3 neighbors for the specified node each time for 5 times
for i in range(5):
    sampled = sample_neighbors(data, node_id, 3)
    print(sampled)

[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 17, 19, 21, 31]
[13, 6, 31]
[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 17, 19, 21, 31]
[21, 13, 6]
[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 17, 19, 21, 31]
[8, 21, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 17, 19, 21, 31]
[8, 4, 11]
[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 17, 19, 21, 31]
[21, 4, 7]
